In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1302,tt0884726,70000000,18662027,Legends of Oz: Dorothy's Return,Lea Michele|Dan Aykroyd|Patrick Stewart|Hugh D...,Dan St. Pierre,There's trouble in OZ...,"Dorothy wakes up in post-tornado Kansas, only ...",88,Animation|Music|Family,Prana Animation Studios|Summertime Entertainment,6/13/2013,6.1,2013
1054,tt1366365,20000000,16863583,The Cold Light of Day,Henry Cavill|VerÃ³nica Echegui|Sigourney Weave...,Mabrouk El Mechri,Be careful who you trust.,A young American (Cavill) uncovers a conspirac...,93,Action|Thriller,Summit Entertainment|Intrepid Pictures|Film Ri...,4/4/2012,4.8,2012
1572,tt0338526,160000000,300257475,Van Helsing,Hugh Jackman|Kate Beckinsale|Richard Roxburgh|...,Stephen Sommers,The One Name They All Fear.,Famed monster slayer Gabriel Van Helsing is di...,132,Horror|Adventure|Action,Universal Pictures|Stillking Films|Sommers Com...,5/5/2004,5.9,2004
865,tt0268380,59000000,383257136,Ice Age,Ray Romano|John Leguizamo|Denis Leary|Goran Vi...,Chris Wedge|Carlos Saldanha,They came. They thawed. They conquered.,"With the impending ice age almost upon them, a...",81,Animation|Comedy|Family|Adventure,Twentieth Century Fox Film Corporation|Blue Sk...,3/10/2002,6.9,2002
1367,tt0355295,88000000,105316267,The Brothers Grimm,Heath Ledger|Matt Damon|Mackenzie Crook|Roger ...,Terry Gilliam,Eliminating Evil Since 1812.,"Folklore collectors and con artists, Jake and ...",118,Adventure|Fantasy|Action|Comedy|Thriller,The Weinstein Company|Summit Entertainment|Atl...,8/25/2005,5.4,2005


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

data['profit'] = (data['revenue'] - data['budget']) # добавяем колонку 'profit' с прибылью от фильма
data['release_date'] = pd.to_datetime(data['release_date']) # преобразовываем дату релиза

'''Следующая функция пригодится нам многократно для более
читабельного вывода названия фильма: получив на вход строку датафрейма (i)
на выходе мы получаем вывод формата "название_фильма (идентификатор_imdb)". '''
def cleaned_name (i):
    title = str(i.original_title)    # переводим название фильма в строковую величину
    imdb = str(i.imdb_id)            # переводим идентификатор imdb в строковую величину
    i1 = title[title.find(' ') + 4 : title.find('\nName:')]  # "отсекаем" лишние части строки
    i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]     # "отсекаем" лишние части строки
    outcome = ('{} ({})'.format(i1, i2))     # формируем итоговый вывод
    return outcome

display(data.sample(5)) # проверили что получилось

display(data.info(verbose=True, null_counts=True)) # проверяем информацию о колонках датафрейма

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
858,tt1440345,25000000,11790979,This Must Be the Place,Sean Penn|Frances McDormand|Judd Hirsch|Eve He...,Paolo Sorrentino,Never for money. Always for love.,"A bored, retired rock star sets out to find hi...",118,Drama,ARP SÃ©lection|Lucky Red|Indigo Film|Medusa Fi...,2011-10-01,6.3,2011,-13209021
495,tt1028576,35000000,60251371,Secretariat,Diane Lane|John Malkovich|Scott Glenn|Dylan Wa...,Randall Wallace,The Impossible True Story,Housewife and mother Penny Chenery agrees to t...,123,Drama,Walt Disney Pictures|Fast Track Productions|Ma...,2010-08-20,6.6,2010,25251371
127,tt2562232,18000000,103215094,Birdman,Michael Keaton|Zach Galifianakis|Edward Norton...,Alejandro GonzÃ¡lez IÃ±Ã¡rritu,or (The Unexpected Virtue of Ignorance),A fading actor best known for his portrayal of...,119,Drama|Comedy,Worldview Entertainment|New Regency Pictures|T...,2014-10-17,7.3,2014,85215094
1519,tt0406816,70000000,55011732,The Guardian,Kevin Costner|Ashton Kutcher|Sela Ward|Melissa...,Andrew Davis,How Do You Decide Who Lives Or Who Dies?,A high school swim champion with a troubled pa...,139,Action,Contrafilm|Firm Films|Touchstone Pictures|Beac...,2006-09-28,7.0,2006,-14988268
163,tt0365907,28000000,53181600,A Walk Among the Tombstones,Liam Neeson|Dan Stevens|David Harbour|Boyd Hol...,Scott Frank,Some people are afraid of all the wrong things,Private investigator Matthew Scudder is hired ...,113,Crime|Drama|Mystery|Thriller,Double Feature Films|Jersey Films|1984 Private...,2014-09-18,6.2,2014,25181600


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

None

# 1. У какого фильма из списка самый большой бюджет?

In [5]:

i = data[data.budget == data.budget.max()] # находим фильм с максимальным бюджетом с помощью среза

answers['1'] = cleaned_name (i)    

display(answers['1'])

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
i = data[data.runtime == data.runtime.max()]

answers['2'] = cleaned_name (i)   

display(answers['2'])

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
i = data[data.runtime == data.runtime.min()]

answers['3'] = cleaned_name (i)     

display(answers['3'])

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [8]:
answers['4'] = round(data["runtime"].mean())

display(answers['4'])

110

# 5. Каково медианное значение длительности фильмов? 

In [9]:
answers['5'] = round(data["runtime"].median())

display(answers['5'])

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
i = data[data.profit == data.profit.max()]

answers['6'] = cleaned_name (i)  

display(answers['6'])  

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [11]:
i = data[data.profit == data.profit.min()]

answers['7'] = cleaned_name (i)    

display(answers['7'])  

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
answers['8'] = len(data[data.profit > 0])

display(answers['8'])  

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
i = data[data.revenue == data.query('release_year == 2008').revenue.max()]

answers['9'] = cleaned_name (i)   

display(answers['9'])  

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [14]:
i = data[data.profit == data.query('2012<=release_year & release_year<=2014').profit.min()]

answers['10'] = cleaned_name (i)   

display(answers['10'])

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [15]:
'''В нашем датафрейме один фильм может принадлежать к нескольким жанрам, 
перечисленным через разделитель '|', из-за чего мы не можем просто работать с колонкой 'genres'.'''

genres_list = data.genres.str.split('|').sum()  # заносим в общий список все жанры всех фильмов
genres_series = pd.Series(genres_list)          # создаем pd.Series из списка
genres_count = genres_series.value_counts()     # создаем упорядоченную по убыванию серию

answers['11'] = genres_count.index[0]           # первый (максималный) элемент будет ответом

display(answers['11'])

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [16]:
'''Задача решается аналогично предыдущей, только для фильмов с прибылью ('profit') больше нуля'''

genres_list = data[data['profit'] > 0].genres.str.split('|').sum()
genres_series = pd.Series(genres_list) 
genres_count = genres_series.value_counts()

answers['12'] = genres_count.index[0] 

display(answers['12'])

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

Вариант 1:

In [17]:
directors_list = data.director.str.split('|').sum()  # создаем список всех режиссеров

directors = []            # создаем список режиссеров и "очищаем" его от дубликатов
for i in directors_list:
    if i not in directors:
        directors.append(i)

director_revenue = {}     # создаем словарь куда будем вносить имя_режиссера:сумму_сборов
for i in directors:
    director_revenue[i] = data[data.director.str.contains(i)]['revenue'].sum()

    
# создаем pd.Series из словаря и выбираем элемент с максимальным значением
answers['13'] = pd.Series(director_revenue).idxmax() 

display(answers['13'])

'Peter Jackson'

Вариант 2:

In [18]:
# создаем список режиссеров с помощью .explode() и .tolist()
directors = data.director.str.split('|').explode().value_counts().index.tolist()
        
directors_revenue = Counter() # для подсчета суммы сборов применяем счетчик вместо словаря
for i in directors:
    directors_revenue[i] = data.loc[data['director'].str.contains(i)]['revenue'].sum()

answers['13'] = pd.Series(directors_revenue).idxmax() 

display(answers['13'])

'Peter Jackson'

Вариант 3:

In [19]:
directors = data.copy()    # создаем копию исходного датафрейма, с которой будем работать

directors['director'] = directors.director.str.split('|') # преобразуем перечень режиссеров в список
directors=directors.explode('director')  # "разбиваем" список режиссеров для каждого фильма

# группируем режиссеров по сумме прибыли, упорядочив по убыванию
dirs_revenue = directors.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

answers['13'] = dirs_revenue.index[0] # первый (максималный) элемент будет ответом

display(answers['13'])

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [20]:
actions = data[data.genres.str.contains('Action')] # выбираем все экшены из датафрейма 

#создаем упорядоченную по убыванию серию с режиссерами
actions_directors = actions.director.str.split('|').explode().value_counts()

answers['14'] = actions_directors.index[0]        # первый (максималный) элемент будет ответом

display(answers['14'])

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

Вариант 1:

In [21]:
films_2012 = data[data['release_year'] == 2012].copy()  # выбираем фильмы 2012 года 

# создаем список с именами всех актеров из фильмов 2012 года
actors = films_2012.cast.str.split('|').explode().value_counts().index.tolist()

actors_revenue = Counter() # создаем счетчик в котором будем суммировать сборы для актеров
for i in actors:
    actors_revenue[i] = films_2012.loc[data['cast'].str.contains(i)]['revenue'].sum()

# создаем pd.Series из счетчика и выбираем 1й элемент (с максимальным значением)
answers['15'] = pd.Series(actors_revenue).idxmax() 

display(answers['15'])

'Chris Hemsworth'

Вариант 2:

In [22]:
films_2012 = data[data['release_year'] == 2012].copy()  # выбираем фильмы 2012 года 

films_2012['cast'] = films_2012.cast.str.split('|') # преобразуем перечень актеров в список
films_2012=films_2012.explode('cast')   # "разбиваем" список актеров для каждого фильма

# группируем актеров по сумме сборов, упорядочив по убыванию
actors_profit = films_2012.groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

answers['15'] = actors_profit.index[0] # первый (максималный) элемент будет ответом

display(answers['15'])

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?
#### Примечание: в фильмах, где бюджет выше среднего по данной выборке.

In [23]:
# создаем новый датафрейм с фильмами с бюджетом выше среднего
high_b_films = data[data['budget'] > data['budget'].mean()]  

#создаем упорядоченную по убыванию серию с актерами
high_b_actors = high_b_films.cast.str.split('|').explode().value_counts()

answers['16'] = high_b_actors.index[0]   # первый (максималный) элемент будет ответом

display(answers['16'])

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [24]:
# создаем новый датафрейм с фильмами с участием Николаса Кейджа
n_cage = data[data.cast.str.contains('Nicolas Cage')] 

genres_list = n_cage.genres.str.split('|').sum()  # заносим в общий список все жанры фильмов
genres_series = pd.Series(genres_list)            # создаем pd.Series из списка
genres_count = genres_series.value_counts()       # создаем упорядоченную по убыванию серию

answers['17'] = genres_count.index[0]             # первый (максималный) элемент будет ответом

display(answers['17'])

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [25]:
# создаем новый датафрейм с фильмами с от Paramount Pictures
paramount = data[data.production_companies.str.contains('Paramount Pictures')]

# выбираем фильм с наименьшей прибылью
i = paramount[paramount.profit == paramount.profit.min()]

answers['18'] = cleaned_name (i)  

display(answers['18'])  

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [26]:
# группируем года выпуска фильмов по сумме сборов, упорядочив по убыванию
years = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False) 

answers['19'] = years.index[0]    # первый (максималный) элемент будет ответом

display(answers['19'])  

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [27]:
# создаем новый датафрейм с фильмами с от Warner Bros
warner = data[data.production_companies.str.contains('Warner Bros')]

# группируем года выпуска фильмов по сумме прибыли, упорядочив по убыванию
years = warner.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)

answers['20'] = years.index[0]    # первый (максималный) элемент будет ответом

display(answers['20'])  

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [28]:
# создаем список с месяцами релиза всех фильмов используя datetime.month
months = []
for i in range (len(data)):
    months.append(data['release_date'][i].month)

#создаем упорядоченную по убыванию серию с месяцами релиза
months_count = pd.Series(months).value_counts()

# нам нужен первый (максималный) элемент
answer = months_count.index[0]

# создаем список с "пронумерованными" названиями месяцев для "красивого" ответа
month_names= pd.Series(["Январь", "Февраль", "Март", "Апрель", 
                        "Май", "Июнь", "Июль", "Август", 
                        "Сентябрь", "Октябрь", "Ноябрь", "Декабрь"],
                        index = range (1, 13))

answers['21'] = month_names.loc[answer]    

display(answers['21'])

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [29]:
# для решения будем использовать серию months_count, созданную на предыдущем шаге

summer_films = 0
for i in range (6, 9):   # просуммируем кол-во фильмов вышедших в 6, 7 и 8 месяц
    summer_films += months_count.loc[i]
    
answers['22'] = summer_films

display(answers['22'])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [38]:
# создаем запрос с перечислением зимних месяцев
winter = 'release_date.dt.month == 1 \
        | release_date.dt.month == 2 \
        | release_date.dt.month == 12'

#создаем упорядоченную по убыванию серию с режиссерами
winter_directors = data.query(winter).director.str.split('|').explode().value_counts()

answers['23'] = winter_directors.index[0]      # первый (максималный) элемент будет ответом

display(answers['23'])

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [31]:
names = data.copy()    # создаем копию исходного датафрейма, с которой будем работать

# заменяем колонку с названиями фильмов на их длины
names['original_title'] = names.original_title.apply(lambda x: len(x))

# превращаем перечень студий в список
names['production_companies'] = names.production_companies.str.split('|')

# "разбиваем" список студий для каждого фильма
names = names.explode('production_companies')

# создаем упорядоченную по убыванию серию со средними длинами названий фильмов каждой студии
lengths = names.groupby(['production_companies'])['original_title'].mean()\
          .sort_values(ascending = False)

answers['24'] = lengths.index[0]  # первый (максималный) элемент будет ответом

display(answers['24'])

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [32]:
overviews = data.copy()    # создаем копию исходного датафрейма, с которой будем работать

# заменяем колонку с описаниями фильмов на кол-во слов в них
overviews['overview'] = overviews.overview.apply(lambda x: len(x.split()))

# превращаем перечень студий в список
overviews.production_companies = overviews.production_companies.str.split('|')

# "разбиваем" список студий для каждого фильма
overviews = overviews.explode('production_companies')

# создаем упорядоченную по убыванию серию со средними длинами описаний фильмов каждой студии
lengths = overviews.groupby(['production_companies'])['overview'].mean()\
          .sort_values(ascending = False)

answers['25'] = lengths.index[0]  # первый (максималный) элемент будет ответом

display(answers['25'])

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [33]:
# вычисляем 99% квантиль рейтинга фильмов
rate_99=data.quantile(0.99)['vote_average']

# делаем выборку фильмов с рейтингом выше 99% квантиля
best=data[data.vote_average > rate_99]

# в качестве ответа берем список названий фильмов из выборки
answers['26'] =  best.original_title.tolist()

display(answers['26'])

# В получившемся списке фильмов больше чем в вариантах ответа. Остается только
# выбрать вариант, все фильмы которого содержатся в нашем списке

['Inside Out',
 'Room',
 'Interstellar',
 'Guardians of the Galaxy',
 'The Imitation Game',
 'Gone Girl',
 'The Grand Budapest Hotel',
 'Inception',
 'The Dark Knight',
 'The Pianist',
 'The Lord of the Rings: The Return of the King',
 'The Wolf of Wall Street',
 '12 Years a Slave',
 'Memento']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [34]:
actors = data.copy()    # создаем копию исходного датафрейма, с которой будем работать
actors['cast'] = actors.cast.str.split('|')  # превращаем перечень актеров в список

actor_pairs = []        # создаем список куда будем заносить пары актеров

for i in range(len(actors)):
    actor_pairs.extend(combinations(actors.loc[i].cast, 2))
    
pairs_series = pd.Series(actor_pairs) 
pairs_count = pairs_series.value_counts()
pairs_df = pd.DataFrame(pairs_count, columns = ['counts'])
pairs_max = pairs_df[pairs_df.counts == pairs_df.counts.max()]    

# в качестве ответа берем список названий фильмов из выборки
answers['27'] =  pairs_max.index.tolist()

display(answers['27'])

# Мы получили 2 пары актеров, снимавшихся вместе больше других, остается выбрать подходящую

[('Daniel Radcliffe', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson')]

# Submission

In [39]:
# в конце можно посмотреть свои ответы к каждому вопросу
display(answers)

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out',
  'Room',
  'Interstellar',
  'Guardians of the Galaxy',
  'The Imitation Game',
  'Gone Girl',
  'The Grand Budapest Hotel',
  'Inception',
  'The Dark Knight',
  'The Pianist',
  'The Lord of the Rings: The Return of the King',
  'The Wolf of Wall Street',
  '12 Years a Slave',
  'Memento'],
 '27': [('Daniel Radcliffe', 'R

In [42]:
# и убедиться что ни чего не пропустил)
print('Колличество полученных ответов: {} из 27.'.format(len(answers)))

Колличество полученных ответов: 27 из 27.
